In [ ]:
import os, time, re, json, subprocess, sys
import pandas as pd
import importlib.util as il

if None in [il.find_spec('python-ulid'), il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-ulid']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
    
from ulid import ULID
import pyperclip

def gen_ulid():
    return str(ULID.from_timestamp(time.time()))

def convert_coord(c):
    c = str(c)
    j = len(c) - 6
    d = int(c[0:2 + j])
    m = int(c[2 + j:4 + j])
    s = float(c[4 + j:6 + j] + '.' + c[6 + j:])
    q = 1 if j == 0 else -1
    coord = round(q * (d + m / 60 + s / 3600), 6)
    
    return coord

def pprint(dict):
    print(json.dumps(dict, indent=2))

def comma_followed_by_number(s):
    for i, char in enumerate(s[:-1]):
        if char == ',' and s[i+1].isdigit():
            return True
    return False

def extract_table_section_from_file(section_header, filename, offset=0):
    offset *= 3
    section_header = '******* ' + section_header + ' *******'

    downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
    with open(os.path.join(downloads_folder, filename), "r") as file:
        lines = file.readlines()

    extracted_lines = []
    inside_section = False
    end_marker_count = 0

    for line in lines:
        if section_header in line:
            inside_section = True
            extracted_lines.append(line)
            continue

        if inside_section:
            if end_marker_count > offset:
                extracted_lines.append(line)
            # Count lines that are mostly dashes
            if line.strip().startswith('---'):
                end_marker_count += 1
                if end_marker_count >= 3 + offset:
                    break

    return "".join(extracted_lines)

def remove_dash_lines(text):
    cleaned_lines = [
        line for line in text.splitlines()
        if not line.strip().startswith("---")
    ]
    return "\n".join(cleaned_lines)

def convert_pipe_text_to_csv(multi_line_text):
    csv_lines = []
    for line in multi_line_text.splitlines():
        if not line.strip():
            continue
        if '|' not in line:
            continue
        
        fields = [field.strip() for field in line.strip('|').split('|')]
        csv_line = '|'.join(fields)
        csv_lines.append(csv_line)

    return '\n'.join(csv_lines)

def csv_text_to_dataframe(csv_text):
    lines = [line.strip() for line in csv_text.strip().split('\n') if line.strip()]
    
    headers = [h.strip() for h in lines[0].split('|')]
    
    data = []
    for line in lines[1:]:
        fields = [f.strip() for f in line.split('|')]
        data.append(fields)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def read_adaptation_section(section_header, filename, offset=0):
    text = extract_table_section_from_file(section_header, filename, offset)
    text = remove_dash_lines(text)
    text = convert_pipe_text_to_csv(text)
    
    return csv_text_to_dataframe(text)

In [ ]:
# Scratchpad Processing
df = read_adaptation_section('FIX', 'adapt.txt', offset=0)

df = df[(df['SIM Only'] == 'N')]
df = df[['Name', 'Short Name', 'Description']]

sp = {}

for index, row in df.iterrows():
    name = row['Name']
    short_name = row['Short Name']
    desc = row['Description']

    if 'DUMMY' in desc:
        continue

    if ',' in desc:
        if not comma_followed_by_number(desc):
            continue
    
    if len(name) == 3:
        sp[name] = desc
    elif len(short_name) == 3:
        sp[short_name] = desc
        
# print(pprint(sp))

df = read_adaptation_section('INTERFAC_AHO', 'adapt.txt', offset=0)
fixes = sorted(df['Exit Fix'].unique().tolist())

fix_pattern = {}
for fix in fixes:
    if fix in sp:
        print('fix_pattern[\'' + fix + '\'] = \'' + sp[fix] + '\'')
print()

df = read_adaptation_section('FIXPAIRS', 'adapt.txt', offset=0)
fixes2 = sorted(df['Exit Fix'].unique().tolist())

for fix in fixes2:
    if fix in fix_pattern:
        continue
    
    if fix in sp:
        print('fix_pattern[\'' + fix + '\'] = \'' + sp[fix] + '\'')

fix_pattern = dict(sorted(fix_pattern.items(), key=lambda item: (0 if '#' in item[1] else 1, item[1])))
pprint(fix_pattern)

scratchpads = []

for s in fix_pattern:
    p = {}
    p['id'] = gen_ulid()
    p['airportIds'] = []
    p['searchPattern'] = fix_pattern[s]

    if 'AOA' in p['searchPattern']:
        idx = p['searchPattern'].index('AOA ')
        p['minAltitude'] = int(p['searchPattern'][idx + 4:])
        p['searchPattern'] = p['searchPattern'][0:idx - 1]
    elif 'AOB' in p['searchPattern']:
        idx = p['searchPattern'].index('AOB ')
        p['maxAltitude'] = int(p['searchPattern'][idx + 4:])
        p['searchPattern'] = p['searchPattern'][0:idx - 1]
    
    p['template'] = s
    scratchpads.append(p)

scratchpads = sorted(scratchpads, key=lambda x: (not '#' in x['searchPattern'], x['searchPattern']))

pyperclip.copy(scratchpads)

In [ ]:
raise 
# MIA Scratchpad Data
fix_pattern['AAR'] = 'AARPS'
fix_pattern['AGE'] = 'AGERS'
fix_pattern['ALN'] = 'ALTNN'
fix_pattern['ALT'] = 'ALTNN#'
fix_pattern['BE1'] = 'BEECH AOA 101'
fix_pattern['BEE'] = 'BEECH AOB 100'
fix_pattern['BNC'] = 'BNICE'
fix_pattern['BNG'] = 'BNGOS#'
fix_pattern['BNI'] = 'BNICE#'
fix_pattern['BNO'] = 'BNGOS'
fix_pattern['DOL'] = 'DORRL'
fix_pattern['DRL'] = 'DORRL#'
fix_pattern['FEA'] = 'FEALX#'
fix_pattern['FEL'] = 'FEALX'
fix_pattern['FLG'] = 'FLMGO AOB 410'
fix_pattern['FLM'] = 'FLMGO# AOB 410'
fix_pattern['FM2'] = 'FLMGO# AOA 411'
fix_pattern['FM3'] = 'FLMGO AOA 411'
fix_pattern['FOL'] = 'FOLZZ#'
fix_pattern['FOZ'] = 'FOLZZ'
fix_pattern['FR2'] = 'FRSBE# AOA 411'
fix_pattern['FR3'] = 'FRSBE AOA 411'
fix_pattern['FRB'] = 'FRSBE AOB 410'
fix_pattern['FRS'] = 'FRSBE# AOB 410'
fix_pattern['GAB'] = 'GABOW#'
fix_pattern['GAO'] = 'GABOW'
fix_pattern['GLA'] = 'GLADZ'
fix_pattern['GV1'] = 'GWAVA'
fix_pattern['GWA'] = 'GWAVA#'
fix_pattern['HRC'] = 'HROCK'
fix_pattern['HRO'] = 'HROCK#'
fix_pattern['HUC'] = 'HURCN'
fix_pattern['HUR'] = 'HURCN#'
fix_pattern['HUS'] = 'HUSIL'
fix_pattern['KET'] = 'KETLL'
fix_pattern['KLA'] = 'KLADA'
fix_pattern['LI6'] = 'LIFRR AOB 060'
fix_pattern['LI7'] = 'LIFFR AOA 061'
fix_pattern['MAN'] = 'MAYNR'
fix_pattern['MAY'] = 'MAYNR#'
fix_pattern['MEL'] = 'MELLZ'
fix_pattern['NNG'] = 'NNOCE# URSUS'
fix_pattern['NNM'] = 'NNOCE# FUNDI'
fix_pattern['PB3'] = 'T208'
fix_pattern['REG'] = 'REGAE'
fix_pattern['SNA'] = 'SNAPR'
fix_pattern['ZFX'] = 'ZFP FLL'
fix_pattern['ZFZ'] = 'ZFP MIA'

In [ ]:
raise 
# TPA Scratchpad Data
scratchpads = []

sr = {
    'A': ['LAL', 'YOJIX', 'V441', 'T336', 'T343'],
    'B': ['DORMR#', 'TIDES# CIGAR', 'TIDES# FROOT', 'DORMR', 'SYKES', 'CIGAR', 'FROOT'],
    'D': ['DARBS', 'V97 AOB 080', 'SZW'],
    'F': ['GANDY#', 'GANDY', 'PAIRS', 'MURDO', 'SABEE', 'SRQ', 'V579', 'RSW', 'CEXAN', 'CHARO', 'VIOLA'],
    'H': ['CROWD#', 'CROWD', 'HALLR', 'V509', 'FAZES'],
    'M': ['PRICY#', 'MINEE#', 'MINEE', 'PRICY'],
    'N': ['BAYPO#', 'BAYPO', 'CAMJO', 'WILON', 'NITTS AOA 100', 'V441', 'OCF', 'GNV'],
    'R': ['KNEED', 'V152', 'T210', 'ORL', 'VARZE'],
    'U': ['ENDED#', 'TIDES# MOMIE', 'TIDES# CAMJO', 'ENDED', 'LACEN', 'MOMIE', 'T495', 'T489', 'ATTAK', 'CTY'], 
}

for k, v in sr.items():
    for pattern in v:
        s = {}
        s['id'] = gen_ulid()
        s['airportIds'] = []
        s['searchPattern'] = pattern
        s['template'] = k + '###'
        scratchpads.append(s)

pyperclip.copy(json.dumps(scratchpads, indent=2))